In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
!pip install spacy
import spacy


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# ------------------------------------------------------------
# Load the Quora training dataset
# ------------------------------------------------------------
df = pd.read_csv("train.csv")

# ------------------------------------------------------------
# Ensure both question columns are converted to proper Python 3 strings
#
# NOTE:
#  - In Python 3, all strings are already Unicode by default.
#  - However, applying str(x) ensures:
#        * NaN values become empty strings ("nan" → "nan" → useful for preprocessing)
#        * Non-string values (floats/ints) are safely converted
#
# The Python 2 code (unicode()) is commented because Python 3 does not need it.
# ------------------------------------------------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))


In [3]:
df.head(2)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# ------------------------------------------------------------
# Merge question1 and question2 into a single text corpus
# TF-IDF needs one combined list of documents to learn vocabulary.
# ------------------------------------------------------------
questions = list(df['question1']) + list(df['question2'])

# ------------------------------------------------------------
# Create a TF-IDF Vectorizer
# lowercase=False → text is already preprocessed; do not alter it here.
# ------------------------------------------------------------
tfidf = TfidfVectorizer(lowercase=False)

# ------------------------------------------------------------
# Fit TF-IDF model on all questions and compute TF-IDF values
# This learns:
#   - vocabulary (all unique words)
#   - IDF (inverse document frequency) for each word
# ------------------------------------------------------------
tfidf.fit_transform(questions)

# ------------------------------------------------------------
# Build a dictionary that maps:
#   word → its learned IDF value
#
# tfidf.get_feature_names() : list of all vocabulary words
# tfidf.idf_                : corresponding IDF scores
# ------------------------------------------------------------
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))


In [5]:
# ------------------------------------------------------------
# Download and load a large English SpaCy model
# ------------------------------------------------------------
!python -m spacy download en_core_web_lg     # large model with 300-dim vectors
!python -m spacy download en                # small default model

# Load the large English model (has word vectors)
nlp = spacy.load('en_core_web_lg')

# ------------------------------------------------------------
# Test the model by embedding a sample word
# Each token in this model has a 300-dimensional vector
# ------------------------------------------------------------
x = nlp('man')
len(x.vector)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


300

In [9]:
import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm

# Load Spacy model
nlp = spacy.load('en_core_web_lg')

# Example: check vector length
x = nlp("man")
print("Vector length:", len(x.vector))

# -----------------------------
# Compute q1_feats_m embeddings
# -----------------------------

vecs1 = []

for qu1 in tqdm(list(df['question1'])):  
    doc1 = nlp(qu1)

    # Initialize matrix: (num_words x vector_dim)
    mean_vec1 = np.zeros((len(doc1), len(doc1[0].vector)))

    for word1 in doc1:
        vec1 = word1.vector
        
        # Get TF-IDF value or 0 if not present
        idf = word2tfidf.get(str(word1), 0)
        
        # Weighted vector
        mean_vec1 += vec1 * idf
    
    # Average over all words → final sentence embedding
    mean_vec1 = mean_vec1.mean(axis=0)

    vecs1.append(mean_vec1)

# Store in dataframe column
df['q1_feats_m'] = vecs1


Vector length: 300


  1%|▊                                                                                                     | 3430/404290 [01:02<2:01:24, 55.03it/s]


KeyboardInterrupt: 

In [6]:
vecs2 = []

# Iterate through each question in Q2
for qu2 in tqdm(df['question2'], desc="Building Q2 vectors"):
    
    # Process the question with SpaCy (tokenization + vectors)
    doc2 = nlp(qu2)

    # Store weighted word vectors
    word_vecs = []

    # ------------------------------------------------------------
    # For each token in the question:
    #   vector = SpaCy 300-dim embedding
    #   idf    = TF-IDF weight (importance)
    # We multiply them to give more weight to rare but important words.
    # ------------------------------------------------------------
    for token in doc2:
        vec = token.vector                      # SpaCy word embedding (300 dims)
        idf = word2tfidf.get(token.text, 0)     # IDF weight from TF-IDF dict
        word_vecs.append(vec * idf)             # Weighted word vector

    # ------------------------------------------------------------
    # Compute the final question embedding:
    #   - Mean of weighted word vectors (common technique in NLP)
    #   - If no valid tokens (rare case) → fallback to zero vector
    # ------------------------------------------------------------
    if len(word_vecs) == 0:
        mean_vec2 = np.zeros(300)               # 300 dimensions for en_core_web_lg
    else:
        mean_vec2 = np.mean(word_vecs, axis=0)

    # Add result to our list
    vecs2.append(mean_vec2)

# Save list of embeddings as a new column
df['q2_feats_m'] = vecs2


Building Q2 vectors: 100%|███████████████████████████████████████████████████████████████████████████████| 404290/404290 [1:44:32<00:00, 64.45it/s]


In [7]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [10]:
# Drop raw question text and IDs from NLP features dataframe
df1 = dfnlp.drop(['qid1', 'qid2', 'question1', 'question2'], axis=1)

# Drop IDs, raw questions, and target variable from preprocessed feature dataframe
df2 = dfppro.drop(['qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], axis=1)

# Drop IDs, raw questions, and target variable from main feature dataframe
df3 = df.drop(['qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], axis=1)

# Expand list/array features of q1 into separate dataframe columns
#df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index=df3.index)

# Expand list/array features of q2 into separate dataframe columns
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index=df3.index)


In [11]:
dfnlp.head(2)

,id,qid1,qid2,question1,question2,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,0.999980,0.833319,0.999983,0.999983,...,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,0.799984,0.399996,0.749981,0.599988,...,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154


In [12]:
df1.head(2)

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154


In [13]:
dfppro.head(2)

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,4,1,51,88,8,13,4.0,20.0,0.200000,5,3


In [14]:
df2.head(2)

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,4,1,51,88,8,13,4.0,20.0,0.200000,5,3


In [15]:
df.head(2)

,id,qid1,qid2,question1,question2,is_duplicate,q2_feats_m
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[0.030659923793279455, 1.076277478691036, -0.0..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[0.2324843898602893, 0.49874637066348715, 1.01..."


In [16]:
df3.head(2)

,id,q2_feats_m
0,0,"[0.030659923793279455, 1.076277478691036, -0.0..."
1,1,"[0.2324843898602893, 0.49874637066348715, 1.01..."


In [17]:
df3_q1.head(2)

NameError: name 'df3_q1' is not defined

In [18]:
df3_q2.head(2)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.030660,1.076277,-0.038813,0.711879,1.069726,-0.367515,-0.405725,-0.015478,0.380043,10.364304,...,-1.370034,0.556233,0.117784,-0.579140,0.036446,-0.912666,-0.868753,0.143559,0.278157,0.920854
1,0.232484,0.498746,1.016505,-0.022019,-0.906428,-0.099234,0.245323,-1.389892,-1.063132,4.848265,...,1.150791,0.171773,-0.258480,0.355125,1.725844,0.308884,-1.388543,0.646322,-0.228228,1.695992


In [22]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
#print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
#print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
#print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
